In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [2]:
url = 'https://www.giants.com/team/stats/'
response = requests.get(url)

In [3]:
print("Response:", response.status_code,response.url) #200 means it went through


Response: 200 https://www.giants.com/team/stats/


In [4]:
giants_stats = BeautifulSoup(response.text,'html.parser')
giants_stats_body = giants_stats.body
nfl_div = giants_stats_body.find_all('div')
giants_stats_body_main_content = giants_stats_body.find(id="main-content")
# print(giants_stats_body_main_content.prettify())

In [28]:
offensive_passing_table = giants_stats_body_main_content.find("div", {"class": "nfl-o-teamstats"})
# print(offensive_passing_table.prettify())

In [6]:
headers = offensive_passing_table.find('thead')
print(headers)

<thead>
<tr>
<th>Player</th>
<th scope="col">ATT</th>
<th scope="col">COMP</th>
<th scope="col">YDS</th>
<th scope="col">COMP%</th>
<th scope="col">YDS/ATT</th>
<th scope="col">TD</th>
<th scope="col">TD%</th>
<th scope="col">INT</th>
<th scope="col">INT%</th>
<th scope="col">LONG</th>
<th scope="col">SCK</th>
<th scope="col">SCK/LOST</th>
<th scope="col">RATE</th>
</tr>
</thead>


In [7]:
clean_headers = headers.find_all('th')
print(clean_headers)

[<th>Player</th>, <th scope="col">ATT</th>, <th scope="col">COMP</th>, <th scope="col">YDS</th>, <th scope="col">COMP%</th>, <th scope="col">YDS/ATT</th>, <th scope="col">TD</th>, <th scope="col">TD%</th>, <th scope="col">INT</th>, <th scope="col">INT%</th>, <th scope="col">LONG</th>, <th scope="col">SCK</th>, <th scope="col">SCK/LOST</th>, <th scope="col">RATE</th>]


In [8]:
columnHeader = []

for c in clean_headers:
    columnHeader.append(c.text)

print(columnHeader)

['Player', 'ATT', 'COMP', 'YDS', 'COMP%', 'YDS/ATT', 'TD', 'TD%', 'INT', 'INT%', 'LONG', 'SCK', 'SCK/LOST', 'RATE']


In [117]:
rowData = []

# clean_offensive_passing_stats = offensive_passing_table.find('tr',{"data-loadmore":"Passing"})

clean_offensive_passing_stats = offensive_passing_table.find_all('td')

# for s in 

# rowData = [s.strip('\n') for x in rowData]

for i in range(0,len(clean_offensive_passing_stats)):
    print(clean_offensive_passing_stats[i])
    # clean_offensive_passing_stats = offensive_passing_table.find('td')
    i = i + 1


print(clean_offensive_passing_stats)
print(i)
# print(clean_offensive_passing_stats)

# rowTable = clean_offensive_passing_stats.find_all('td')

<td class="sorter-lastname">
<div class="d3-o-media-object">
<a href="/team/players-roster/alex-tanney/">
<figure class="d3-o-media-object__figure">
<picture is-lazy="/t_lazy"><!--[if IE 9]><video style="display: none; "><![endif]--><source media="(min-width:1024px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/giants/vfw4txeghdoarfkpottn 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/giants/vfw4txeghdoarfkpottn 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/giants/vfw4txeghdoarfkpottn"/><source media="(min-width:768px)" srcset="https://static.clubs.nfl.com/image/private/t_thumb_squared/t_lazy/f_auto/giants/vfw4txeghdoarfkpottn 1x, https://static.clubs.nfl.com/image/private/t_thumb_squared_2x/t_lazy/f_auto/giants/vfw4txeghdoarfkpottn 2x, https://static.clubs.nfl.com/image/private/t_thumb_squared_3x/t_lazy/f_auto/giants/vfw4txeghdoarfkpottn"/><source srcset="https://static.clubs.nfl.com

In [10]:
# rowData = []


# for r in clean_offensive_passing_stats:
#     rowData.append(r.text)
#     # str(rowData).split


# rowData = [r.strip('\n') for r in rowData]
# # clean_rowData = str(rowData).split('\n')
# clean_rowData


# # print(rowData)
# # print()
# # rowData = [r.strip('\n') for r in rowData]
# # print()
# # print(rowData)
# # rowData = [r.strip('\n\n') for r in rowData]
# # print()
# # print(rowData)



["['Alex Tanney\\n\\n\\n\\n                        1                      \\n\\n                        1                      \\n\\n                        1                      \\n\\n                        100                      \\n\\n                        1                      \\n\\n                        0                      \\n\\n                        0                      \\n\\n                        0                      \\n\\n                        0                      \\n\\n                        1                      \\n\\n                        0                      \\n\\n                        0                      \\n\\n                        79.2                      ', 'Eli Manning\\n\\n\\n\\n                        147                      \\n\\n                        91                      \\n\\n                        1042                      \\n\\n                        61.9                      \\n\\n                        7.1          